# Import de librerías y preparación de data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import glob
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
ruta = "/content/drive/My Drive/Maestria/Mineria de datos/Dataset proyecto/"

In [21]:
archivos = glob.glob(os.path.join(ruta, "*.xlsx"))
lista_df = [pd.read_excel(f) for f in archivos]
datos = pd.concat(lista_df, ignore_index=True)

In [ ]:
datos.head()

,AÑO,MES,DIASESTAN,SEXO,PPERTENENCIA,EDAD,PERIODOEDA,DEPTORESIDEN,MUNIRESIDEN,CAUFIN,CONDIEGRES,TRATARECIB,DIASESTANCIA
0,2024,1,1.0,2,1,5,2,1,101,A010,1,1,NaN
1,2024,1,1.0,2,1,1,3,1,101,A010,1,1,NaN
2,2024,1,2.0,1,4,3,3,1,101,A010,1,1,NaN
3,2024,1,2.0,2,1,6,3,1,101,A010,1,1,NaN
4,2024,2,3.0,1,9,6,3,1,101,A010,1,1,NaN


In [ ]:
# Filtrado similar al de R
datos_nn1 = datos.copy()
datos_nn1 = datos_nn1[
    (datos_nn1["EDAD"].between(0, 99)) &
    (datos_nn1["DIASESTAN"].between(1, 98))
]

# Crear variable objetivo TRATA_TIPO (SIMPLE / COMPLEJO)
def clasificar_trata_tipo(x):
    if x == 1:
        return "SIMPLE"
    elif x in [2, 3]:
        return "COMPLEJO"
    else:
        return np.nan

datos_nn1["TRATA_TIPO"] = datos_nn1["TRATARECIB"].apply(clasificar_trata_tipo)

In [ ]:
# Eliminar registros sin clase
datos_nn1 = datos_nn1.dropna(subset=["TRATA_TIPO"])

# Variables de entrada y salida
X1 = datos_nn1[["EDAD", "SEXO", "PPERTENENCIA", "DIASESTAN"]].copy()
y1 = datos_nn1["TRATA_TIPO"].copy()

# Codificar la salida como 0/1
mapa_trata1 = {"SIMPLE": 0, "COMPLEJO": 1}
y1_bin = y1.map(mapa_trata1).values

# Normalizar X
scaler1 = StandardScaler()
X1_scaled = scaler1.fit_transform(X1)

# División train / test
X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1_scaled, y1_bin, test_size=0.2, random_state=42
)


In [ ]:
# Construcción del modelo RN1 (binaria)
model1 = Sequential()
model1.add(Dense(32, activation='relu', input_dim=X1_train.shape[1]))
model1.add(Dense(16, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))  # salida binaria

model1.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\n=== Entrenando RN1 – Tratamiento SIMPLE vs COMPLEJO ===")
history1 = model1.fit(
    X1_train, y1_train,
    epochs=20,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



=== Entrenando RN1 – Tratamiento SIMPLE vs COMPLEJO ===
Epoch 1/20
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.8288 - loss: 0.4139 - val_accuracy: 0.8555 - val_loss: 0.3569
Epoch 2/20
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8546 - loss: 0.3572 - val_accuracy: 0.8577 - val_loss: 0.3500
Epoch 3/20
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8542 - loss: 0.3546 - val_accuracy: 0.8576 - val_loss: 0.3496
Epoch 4/20
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8570 - loss: 0.3490 - val_accuracy: 0.8579 - val_loss: 0.3478
Epoch 5/20
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8564 - loss: 0.3492 - val_accuracy: 0.8579 - val_loss: 0.3491
Epoch 6/20
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8545 - loss: 0.3519 - val_accuracy: 0.8580 - val_loss: 0.3476
Epoch 7/20
1728/1728 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8543 - loss: 0.3517 - val_accuracy: 0.8579 - val_loss: 0.3482
Epoch 8/20
1728/1728 ━━━━━━━━━━━━

In [ ]:
# Evaluación RN1
loss1, acc1 = model1.evaluate(X1_test, y1_test, verbose=0)
print(f"Precisión RN1 en test: {acc1:.4f}")
print(f'Loss: {loss1}, Accuracy: {acc1}')

Precisión RN1 en test: 0.8548
Loss: 0.3505362570285797, Accuracy: 0.8547901511192322


# Predicciones de escenarios

In [ ]:
esc1_A = pd.DataFrame({
    "EDAD": [22],
    "SEXO": [2],
    "PPERTENENCIA": [1],
    "DIASESTAN": [2]
})

esc1_B = pd.DataFrame({
    "EDAD": [22],
    "SEXO": [2],
    "PPERTENENCIA": [2],
    "DIASESTAN": [1]
})

esc1_C = pd.DataFrame({
    "EDAD": [10],
    "SEXO": [2],
    "PPERTENENCIA": [3],
    "DIASESTAN": [1]
})

esc1_D = pd.DataFrame({
    "EDAD": [65],
    "SEXO": [1],
    "PPERTENENCIA": [1],
    "DIASESTAN": [4]
})

In [ ]:
def predecir_rn1(escenario_df, nombre):
    esc_scaled = scaler1.transform(escenario_df)
    prob = model1.predict(esc_scaled)[0][0]
    clase = "COMPLEJO" if prob >= 0.5 else "SIMPLE"
    print(f"\nEscenario {nombre} – RN1")
    print(f"Probabilidad de COMPLEJO: {prob:.3f}")
    print(f"Clasificación final: {clase}")

predecir_rn1(esc1_A, "A")
predecir_rn1(esc1_B, "B")
predecir_rn1(esc1_C, "C")
predecir_rn1(esc1_D, "D")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step

Escenario A – RN1
Probabilidad de COMPLEJO: 0.853
Clasificación final: COMPLEJO
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step

Escenario B – RN1
Probabilidad de COMPLEJO: 0.592
Clasificación final: COMPLEJO
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step

Escenario C – RN1
Probabilidad de COMPLEJO: 0.252
Clasificación final: SIMPLE
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step

Escenario D – RN1
Probabilidad de COMPLEJO: 0.147
Clasificación final: SIMPLE
